In [ ]:
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune import register_env

import rllib_setup

ray.init(ignore_reinit_error=True)


In [ ]:
env_name = "VJS20_6"
register_env(
    env_name,
    lambda config: rllib_setup.get_env_continuous(20,6),
)
test_env = rllib_setup.get_env_continuous(20,6)


def policies(agent_ids):
    obs_space = test_env.observation_space
    act_space = test_env.action_space
    return {
        str(i): (
            None,
            obs_space,
            act_space,
            {}
            # config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }


In [ ]:
config = (
    PPOConfig().rollouts(num_rollout_workers=48, batch_mode="complete_episodes")
    .resources(num_gpus=2, num_cpus_for_local_worker=50)
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, **kwargs: str(agent_id),
    )
    .environment(disable_env_checking=True).evaluation(evaluation_interval=10)
)
# print(config.to_dict())
# Build a Algorithm object from the config and run one training iteration.
algo = config.build(env=env_name)


(autoscaler +55m34s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(autoscaler +56m10s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


In [ ]:
algo.train()


In [ ]:
algo.evaluate()


In [ ]:
from ray import tune


In [ ]:
alg_name = "PPO"
tune.run(
    alg_name,
    name="PPO",
    stop={"episodes_total": 10000},
    checkpoint_freq=10,
    config=config.to_dict(),
)


In [ ]:
for j in range(10):
    for i in range(10):
        info = algo.train()
        print(info['info']['learner']['Machine_1']['learner_stats']['total_loss'])
